## PERIN

Preprocess the data from the stancer pipeline and bring it into canonical form so it can be used by the sentiment graph parser.

In [1]:
# IMPORTS
import pandas as pd
import json
import os

In [2]:
# makedirs if not exist
os.makedirs("../../etl/data/processed/Perin_Preprocessing", exist_ok=True)

In [3]:
TRAIN_INPUT_PATH = "../../etl/data/intermediate/TrainTestSplit/01_train.csv"
TEST_INPUT_PATH = "../../etl/data/intermediate/TrainTestSplit/01_test.csv"
VAL_INPUT_PATH = "../../etl/data/intermediate/TrainTestSplit/01_val.csv"

TRAIN_OUTPUT_PATH = "../../etl/data/processed/Perin_Preprocessing/01_train.json"
TEST_OUTPUT_PATH = "../../etl/data/processed/Perin_Preprocessing/01_test.json"
VAL_OUTPUT_PATH = "../../etl/data/processed/Perin_Preprocessing/01_val.json"

MANANN_INPUT_PATH = "../../etl/data/intermediate/TrainTestSplit/manual_annotation.csv"
MANANN_OUTPUT_PATH = "../../etl/data/processed/Perin_Preprocessing/manual_annotation.json"

GOLD_STANDARD_INPUTPATH = "../../etl/data/intermediate/ReadGoldStandard/pro_con_500.csv"
GOLD_STANDARD_OUTPUTPATH = "../../etl/data/processed/Perin_Preprocessing/pro_con_500.json"

# Full phrase or Head mode
FULL_PHRASE = False
USE_FULL_SENTENCE = True
INCLUDE_MANANN = False
INCLUDE_GOLD = False

In [4]:
# read in each dataframe
input_paths = [TRAIN_INPUT_PATH, TEST_INPUT_PATH, VAL_INPUT_PATH]

if INCLUDE_MANANN:
    input_paths.append(MANANN_INPUT_PATH)
elif INCLUDE_GOLD:
    input_paths.append(GOLD_STANDARD_INPUTPATH)

dataframes = []

for ip in input_paths:
    df = pd.read_csv(ip, index_col=0)
    dataframes.append(df)


In [5]:
# inspect a df
dataframes[1]

,verb_form,verb_form_start,verb_form_end,verb_lemma,arg1,arg1_start,arg1_end,arg1_pos,arg1_head,arg1_head_start,...,arg2,arg2_start,arg2_end,arg2_pos,arg2_head,arg2_head_start,arg2_head_end,rel_type,pred_serial,full_sentence_text
doc_id,,,,,,,,,,,,,,,,,,,,,
ab63df5f2fafbd3de6030b65f3c3dd1d0fd3a1dc25ee421f7062733986df8e0f,bestraft,61,69,bestrafen,Rechtzeitiges Zahlen,0,20,N,Zahlen,14,...,Geldausgaben,36,48,N,Geldausgaben,36,48,con,"Predicate(type='con', args=(Head(sentence=38, ...","Rechtzeitiges Zahlen wird belohnt , Geldausgab..."
ed925a3aec898b06f61ee9cbbe5603b9f77b02d8d4a38af84332f5ca6b752692,demotiviert,168,179,demotivieren,die Strategie der « Entdiabolisierung,49,86,N,Strategie,53,...,einen Teil ihrer Wählerschaft,138,167,N,Teil,144,148,con,"Predicate(type='con', args=(Head(sentence=8, t...","Die Regionalwahlen Ende Juni haben belegt , da..."
a1eb8f7b36bb4aece2d86d6a48958a2327d003dbe1daf15ef81f16da040d0a81,fehlt,21,26,fehlen,die Unabhängigkeit,27,45,N,Unabhängigkeit,31,...,Vielen Delegierten,2,20,N,Delegierten,9,20,con,"Predicate(type='con', args=(Head(sentence=6, t...",« Vielen Delegierten fehlt die Unabhängigkeit ...
da30dfefee97fef8b9b24d8423f1db1b4ef24ffd66db8203017786c588017443,getäuscht,104,113,täuschen,Man City,81,89,N,Man,81,...,die Uefa,95,103,N,Uefa,99,103,con,"Predicate(type='con', args=(Head(sentence=3, t...",Im Rahmen der Football Leaks berichtete das Na...
467703983b5eaf4b9c71d25008b19a5325960ebe94bb6b0cfa918f6f84072c5b,beschwert,79,88,beschweren,verschiedene Anwohnerinnen und Anwohner,25,64,N,Anwohnerinnen,38,...,den Lärm,70,78,N,Lärm,74,78,con,"Predicate(type='con', args=(Head(sentence=10, ...",In Schlieren hatten sich verschiedene Anwohner...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4ce28e59a2fbc452fe70e9f3f5ae3ec972a6f576604087c8c57a9c67356b2f95,feierte,20,27,feiern,Brägger,28,35,N,Brägger,28,...,Den grössten Erfolg,0,19,N,Erfolg,13,19,pro,"Predicate(type='pro', args=(Head(sentence=42, ...",Den grössten Erfolg feierte Brägger an den Eur...
b269d191fdcfad4e762d5650b0f8afeb5614c858167720d8f54a1eb84fad746a,stellt,33,39,stellen,das Ensi,45,53,N,Ensi,49,...,diesen Standpunkt jedenfalls,4,32,N,Standpunkt,11,21,pro,"Predicate(type='pro', args=(Head(sentence=14, ...",Auf diesen Standpunkt jedenfalls stellt sich d...
202f8d2a34760d33ff3bbbdc901475ae10a7fbc4dfe919abadc66d6368e1b093,sorgt,106,111,sorgen,Pro Senectute,92,105,N,Pro,92,...,die Senioren,120,132,N,Senioren,124,132,pro,"Predicate(type='pro', args=(Head(sentence=3, t...","Doch zumindest ein Kanton bremst , der Konsume..."


In [6]:
dataframes[1].rel_type.value_counts()

neutral    164
pro        161
con        147
Name: rel_type, dtype: int64

In [7]:
def transform_to_semeval_ssa_format(df):
    sent_list = []
    sentences = list(set(df.full_sentence_text.to_list()))
    for i, sentence in enumerate(sentences):
        opinions_list = []
        sub_df = df[df["full_sentence_text"] == sentence]
        for row in sub_df.itertuples():
            # skip neutral case, not needed here
            # if row.rel_type == "neutral":
            #    continue

            if FULL_PHRASE:
                # only-NPs
                opinions_list.append({
                    "Source": [[row.arg1], [f"{row.arg1_start}:{row.arg1_end}"]],
                    "Target": [[row.arg2], [f"{row.arg2_start}:{row.arg2_end}"]],
                    "Polar_expression": [[row.verb_form], [f"{row.verb_form_start}:{row.verb_form_end}"]],
                    "Polarity": "Positive" if row.rel_type == "pro" else "Negative" if row.rel_type == "con" else "Neutral",
                    # we currently do not care about intensity
                    "Intensity": "Average"
                })
            else:
                # non-NPs
                opinions_list.append({
                    "Source": [[row.arg1_head], [f"{row.arg1_head_start}:{row.arg1_head_end}"]],
                    "Target": [[row.arg2_head], [f"{row.arg2_head_start}:{row.arg2_head_end}"]],
                    "Polar_expression": [[row.verb_form], [f"{row.verb_form_start}:{row.verb_form_end}"]],
                    "Polarity": "Positive" if row.rel_type == "pro" else "Negative" if row.rel_type == "con" else "Neutral",
                    # we currently do not care about intensity
                    "Intensity": "Average"
                })

        if len(opinions_list) != 0:
            sent_list.append({
                "sent_id": str(i),
                "text": sentence,
                "opinions": opinions_list
            })
        else:
            continue
    return sent_list

In [8]:
# test the function
data = transform_to_semeval_ssa_format(dataframes[1])

In [9]:
# verify data
data[:1]

[{'sent_id': '0',
  'text': 'Immer wieder schütteln Anschläge Afghanistans Hauptstadt durch , vergangenes Jahr wurde eine Universität angegriffen , 32 Personen getötet .',
  'opinions': [{'Source': [['Universität'], ['93:104']],
    'Target': [['Personen'], ['122:130']],
    'Polar_expression': [['getötet'], ['131:138']],
    'Polarity': 'Negative',
    'Intensity': 'Average'}]}]

In [10]:
FULL_PHRASE

False

In [11]:
x = "Und auch die unabhängigen Verwaltungsräte sowie Konzernleitung und Management stellten sich quer und rebellierten gegen die unerwünschte Übernahme ."
eval('x[' + '26:41' + ']')

'Verwaltungsräte'

## Minimal Sequence Dataset

The idea here is to see whether a pruning approach, that is learning to make the sentence shorter through syntactic dependency reduction would yield an improved result.

This is, would learning to prune help? https://aclanthology.org/D18-1244/

In [12]:
import copy

def convert_ssa_to_minimal_sequence_data(data):
    data = copy.deepcopy(data)
    ret_data = []
    for sent in data:
        opinions = []
        minimal_sent = ""
        for opinion in sent["opinions"]:
            try:
                # print("OsS", opinion["Source"][1][0])
                minimal_sent += eval('sent["text"][' + opinion["Source"][1][0] + ']') + " "
                # print(opinion["Source"])
                opinion["Source"][1][0] = str(minimal_sent.find(opinion["Source"][0][0])) + ":" + str(len(opinion["Source"][0][0]))

                minimal_sent += eval('sent["text"][' + opinion["Target"][1][0] + ']') + " "
                # print(eval('sent["text"][' + opinion["Target"][1][0] + ']') + " ")
                # print(minimal_sent)
                # print(opinion["Target"])
                opinion["Target"][1][0] = str(minimal_sent.find(opinion["Target"][0][0])) + ":" + str(len(opinion["Target"][0][0]))

                minimal_sent += eval('sent["text"][' + opinion["Polar_expression"][1][0] + ']') + " "
                opinion["Polar_expression"][1][0] = str(minimal_sent.find(opinion["Polar_expression"][0][0])) + ":" + str(len(opinion["Polar_expression"][0][0]))
                opinions.append(opinion)
            except:
                continue
        if len(opinions) > 0:
            sent["opinions"] = opinions
            # add a dot to every sentence at the end, possibly fixes problem
            sent["text"] = minimal_sent + "."
            ret_data.append(sent)
        else:
            continue
    return ret_data

In [13]:
# data_minimal

In [14]:
output_paths = [TRAIN_OUTPUT_PATH, TEST_OUTPUT_PATH, VAL_OUTPUT_PATH, MANANN_OUTPUT_PATH, GOLD_STANDARD_OUTPUTPATH]

for op, df in zip(output_paths, dataframes):
    # call function for each... dev, test, train-json
    data = transform_to_semeval_ssa_format(df)
    # convert to JSON format.
    if not USE_FULL_SENTENCE:
        data = convert_ssa_to_minimal_sequence_data(data)
    with open(op, mode="w") as f:
        json.dump(data, f, ensure_ascii=False)

In [15]:
USE_FULL_SENTENCE

True